# Proyecto (análisis dataset)

In [1]:
# --- General ---
import os

# --- Data ---
import pandas as pd
# from pandasql import sqldf
# import numpy as np
# import statistics as stats

# --- Conexión ---
# import elasticsearch

# --- Procesamiento lenguaje: spacy ---
import spacy
from spacy.matcher import Matcher
# from spacy.matcher import PhraseMatcher

# --- Procesamiento lenguaje: gensim ---
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel

# --- Prediccion Positividad: POS, NEU, NEG ---
from tqdm import tqdm
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# --- Visualización ---
import pyLDAvis
import pyLDAvis.gensim_models 
# import matplotlib.colors as colors
# import matplotlib.pyplot as plt
# import geopandas as gpd
%matplotlib inline

# ==============================================================================

# --- Configuración ---
# User password
password = os.environ.get('SOPHIA2')

# - - - geoData - - -
# Import tools_region.py
import tools_region as tr

# --- Funciones ---
# Cargar paquete de español mediano
nlp = spacy.load("es_core_news_md")

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

c:\Users\darko\Miniconda3\envs\myenv\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


### Utilización de regiones y comunas

In [3]:
regiones = tr.get_regiones()
tr.list_regions(regiones)

Arica_y_Parinacota
Tarapacá
Antofagasta
Atacama
Coquimbo
Valparaíso
Región_del_Libertador_Gral._Bernardo_O’Higgins
Región_del_Maule
Región_de_Ñuble
Región_del_Biobío
Región_de_la_Araucanía
Región_de_Los_Ríos
Región_de_Los_Lagos
Región_Aisén_del_Gral._Carlos_Ibáñez_del_Campo
Región_de_Magallanes_y_de_la_Antártica_Chilena
Región_Metropolitana_de_Santiago


In [4]:
data_regiones_comunas = tr.search_region('Región_de_Los_Lagos')
if(data_regiones_comunas):
    region = data_regiones_comunas['region']
    comunas = data_regiones_comunas['comunas']

Region encontrada


## Leemos la información de los archivos

In [6]:
# --- Cargar Archivo ---
nombre_archivo = 'data_Region_X' # Resultado: ./data/{nombre_archivo}_{from_}_{to_}.csv
from_="2022-01-01"
to_="2022-01-31"

In [7]:
nombre_archivo = nombre_archivo+"_"+from_+"_"+to_
df = pd.read_csv('./data/'+nombre_archivo+'.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)

df.head(2)

,id_news,country,media_outlet,url,title,text,date,San_Pablo,San_Juan_de_la_Costa,Río_Negro,...,Frutillar,Fresia,Dalcahue,Curaco_de_Vélez,Cochamó,Chonchi,Chaitén,Castro,Calbuco,Ancud
0,21907946.0,chile,radiosago,https://www.radiosago.cl/investigan-muerte-de-...,Investigan muerte de hombre apuñalado en Máfil...,La Fiscalía de Los Lagos dirige una investigac...,2022-01-26,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,21908053.0,chile,radiosago,https://www.radiosago.cl/ante-retroceso-a-fase...,Ante retroceso a Fase 3: Municipalidad de Puye...,Frente al alza de contagios de Covid-19 en la ...,2022-01-24,0,1,1,...,0,0,0,0,0,0,0,0,0,0


## Definimos los patrones de búsqueda

In [8]:
# --- Texto de las noticias a lista ---
noticias = df.text.values.tolist()

nlp = spacy.load("es_core_news_sm")
matcher = Matcher(nlp.vocab)

# --- Definir patrones ---
# pattern: NOUN-de-NOUN
pattern_1 = [{"POS": "NOUN"},{"LOWER": "de"}, {"POS": "NOUN"}]
matcher.add("NOUN-de-NOUN", [pattern_1])

# pattern: NOUN-ADJ
pattern_2 = [{"POS": "NOUN"}, {"POS": "ADJ"}]
matcher.add("NOUN-ADJ", [pattern_2])

## Creamos filtro palabras relevantes

In [9]:
def text_to_list(noticia):
    list_of_words = []
    
    try:
        doc = nlp(noticia)

        for token in doc:
            if (token.pos_=="NOUN"):
                list_of_words.append(token.text)

        for ent in doc.ents:
            if (ent.label_ == "PER" and " " in ent.text):
                list_of_words.append(ent.text)

        matches = matcher(doc)

        for match_id, start, end in matches:
            span = doc[start:end]  # The matched span
            list_of_words.append(span.text)
    
    except Exception as e: 
        print(noticia)
        print(e)
    
    return list_of_words

## Procesamos las noticias

In [10]:
# --- Procesar texto ---
noticias_procesadas = []
for index, noticia in enumerate(noticias):
    noticia_procesada = text_to_list(noticia)
    noticias_procesadas.append(noticia_procesada)

## Prepararamos los datos de entrada de LDA
Los datos de entrada de LDA son: 
- un diccionario (variable 'id2word')
- nuestro dataset preprocesado ('noticias_procesadas')

In [11]:
# --- Crear diccionario de palabras ---
id2word = corpora.Dictionary(noticias_procesadas)
texts = noticias_procesadas

dataset_id = [id2word.doc2bow(noticia_procesada) for noticia_procesada in noticias_procesadas]

In [12]:
# --- Crear modelo LDA ---
lda_model = gensim.models.ldamodel.LdaModel(corpus=dataset_id,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [13]:

# --- Generar el mejor modelo LDA ---
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        print(num_topics)
        
        model = gensim.models.ldamodel.LdaModel(corpus=dataset_id,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
        
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [14]:
start=2
limit=10
step=2

In [15]:
# --- Optimal model ---
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=dataset_id, texts=noticias_procesadas, start=start, limit=limit, step=step)

2
4
6
8


In [16]:
# get max value
max_coh_value = -1
for index, coh_value  in enumerate(coherence_values):
    if coh_value > max_coh_value:
        max_coh_value = coh_value
        max_coh_index = index

In [17]:
print(f'Numero de topicos: {start + max_coh_index*step}')
optimal_model = model_list[max_coh_index]

Numero de topicos: 8


## Visualizar tópicos

In [18]:
# Visualize the topics
pyLDAvis.enable_notebook();
vis = pyLDAvis.gensim_models.prepare(optimal_model, dataset_id, id2word);
vis

/home/alex/miniconda3/envs/myenv/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/home/alex/miniconda3/envs/myenv/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/alex/miniconda3/envs/myenv/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/alex/miniconda3/envs/myenv/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/alex/minico

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.301239 -0.005626       1        1  45.102004
5     -0.164202  0.107049       2        1  18.110620
2     -0.005794 -0.291126       3        1  14.263868
1     -0.017494  0.063348       4        1   9.482725
7      0.097342  0.039731       5        1   5.330738
0      0.144804  0.032858       6        1   3.486629
6      0.125481  0.038283       7        1   3.162304
3      0.121102  0.015483       8        1   1.061111, topic_info=                 Term        Freq       Total Category  logprob  loglift
963          proyecto  434.000000  434.000000  Default  30.0000  30.0000
56              casos  186.000000  186.000000  Default  29.0000  29.0000
74           personas  803.000000  803.000000  Default  28.0000  28.0000
1682             agua  305.000000  305.000000  Default  27.0000  27.0000
168            región  463.000000  463.000000  Default  26.0000  26.0000
...               ...         ...         ...      ...      ...      ...
2678      gastronomía    2.943008   10.747101   Topic8  -6.1483   3.2506
501         denuncias    4.636468   37.475073   Topic8  -5.6938   2.4561
3220  Francisco Muñoz    2.052391    4.452046   Topic8  -6.5087   3.7715
2236         promedio    3.083078   59.110457   Topic8  -6.1018   1.5924
1000        temporada    1.919586   56.774173   Topic8  -6.5756   1.1589

[413 rows x 6 columns], token_table=       Topic      Freq             Term
term                                   
1011       4  0.978696    Carlos Geisse
2988       7  0.908186       Detectives
10183      7  0.951794      Estudiantes
3220       4  0.449232  Francisco Muñoz
3220       8  0.449232  Francisco Muñoz
...      ...       ...              ...
185        4  0.107315             zona
3295       1  0.292210            áreas
3295       3  0.639210            áreas
3295       5  0.063921            áreas
4906       5  0.982854           órgano

[557 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 6, 3, 2, 8, 1, 7, 4])

## Temas principales

In [19]:
def format_topics_documents(ldamodel=None, corpus=dataset_id, texts=texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['dominant_Topic', 'perc_Contribution', 'topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_documents(ldamodel=optimal_model, corpus=dataset_id, texts=noticias)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['document_No', 'dominant_Topic', 'topic_Perc_Contrib', 'keywords', 'text']
df_dominant_topic.head(5)

/tmp/ipykernel_11940/2884501078.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
/tmp/ipykernel_11940/2884501078.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)


,document_No,dominant_Topic,topic_Perc_Contrib,keywords,text
0,0,4,0.9553,"personas, años, año, comuna, parte, proceso, e...",La Fiscalía de Los Lagos dirige una investigac...
1,1,4,0.9614,"personas, años, año, comuna, parte, proceso, e...",Frente al alza de contagios de Covid-19 en la ...
2,2,4,0.5943,"personas, años, año, comuna, parte, proceso, e...","La jefa comunal de la comuna lacustre, María J..."
3,3,4,0.8474,"personas, años, año, comuna, parte, proceso, e...",La jefa de la División de Atención Primaria de...
4,4,0,0.5230,"emprendimiento, naturaleza, conservación, cana...",Como una necesidad por plantear la urgencia qu...


In [20]:
# add comuna to dataframe df_dominant_topic using merge
df_dominant_topic = pd.merge(df_dominant_topic, df)

## Revisar temas principales

In [21]:
df_dominant_topic.head(2)

,document_No,dominant_Topic,topic_Perc_Contrib,keywords,text,id_news,country,media_outlet,url,title,...,Frutillar,Fresia,Dalcahue,Curaco_de_Vélez,Cochamó,Chonchi,Chaitén,Castro,Calbuco,Ancud
0,0,4,0.9553,"personas, años, año, comuna, parte, proceso, e...",La Fiscalía de Los Lagos dirige una investigac...,21907946.0,chile,radiosago,https://www.radiosago.cl/investigan-muerte-de-...,Investigan muerte de hombre apuñalado en Máfil...,...,0,0,0,0,0,0,0,0,0,0
1,1,4,0.9614,"personas, años, año, comuna, parte, proceso, e...",Frente al alza de contagios de Covid-19 en la ...,21908053.0,chile,radiosago,https://www.radiosago.cl/ante-retroceso-a-fase...,Ante retroceso a Fase 3: Municipalidad de Puye...,...,0,0,0,0,0,0,0,0,0,0


## Procesaremos la positividad de los datos (POS, NEU, NEG)

In [25]:
# --- Cargamos el modelo ---
model_name = "finiteautomata/beto-sentiment-analysis"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

In [27]:
# --- Funcion para obtener el sentimiento ---
df_dominant_topic['title_label'] = ""
df_dominant_topic['title_score'] = ""

for index, row in tqdm(df_dominant_topic.iterrows(), desc='Análisis de sentimiento', total=df_dominant_topic.shape[0]):
    
    # Analizamos su sentimiento en inglés
    sentiment_value = nlp(row['title'])
    
    # Insertamos en dataframe
    df_dominant_topic.at[index, "title_label"] = sentiment_value[0].get('label')
    df_dominant_topic.at[index, "title_score"] = sentiment_value[0].get('score')

Análisis de sentimiento: 100%|██████████| 1380/1380 [02:30<00:00,  9.20it/s]


In [29]:
df_dominant_topic.head(2)

,document_No,dominant_Topic,topic_Perc_Contrib,keywords,text,id_news,country,media_outlet,url,title,...,Dalcahue,Curaco_de_Vélez,Cochamó,Chonchi,Chaitén,Castro,Calbuco,Ancud,title_label,title_score
0,0,4,0.9553,"personas, años, año, comuna, parte, proceso, e...",La Fiscalía de Los Lagos dirige una investigac...,21907946.0,chile,radiosago,https://www.radiosago.cl/investigan-muerte-de-...,Investigan muerte de hombre apuñalado en Máfil...,...,0,0,0,0,0,0,0,0,NEU,0.537904
1,1,4,0.9614,"personas, años, año, comuna, parte, proceso, e...",Frente al alza de contagios de Covid-19 en la ...,21908053.0,chile,radiosago,https://www.radiosago.cl/ante-retroceso-a-fase...,Ante retroceso a Fase 3: Municipalidad de Puye...,...,0,0,0,0,0,0,0,0,NEG,0.863434


## Guardamos nuevamente el dataset pero ahora procesado

In [30]:
df_dominant_topic.to_csv('./data/'+nombre_archivo+'_dominant_topic.csv', index=False)